<a href="https://colab.research.google.com/github/apriandito/nowcasting/blob/main/1_Getting_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

import requests
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go

Get API Key: http://alphavantage.co/

In [2]:
# Ambil data dari API
API_KEY = '8V7F020H9WXQRHLQ'
url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol=IDR&apikey={API_KEY}&outputsize=full"
response = requests.get(url)
data = response.json()
timeseries = data.get("Time Series FX (Daily)", {})
timeseries

{'2025-05-01': {'1. open': '16674.00000',
  '2. high': '16674.00000',
  '3. low': '16594.00000',
  '4. close': '16597.00000'},
 '2025-04-30': {'1. open': '16782.00000',
  '2. high': '16782.00000',
  '3. low': '16589.00000',
  '4. close': '16594.00000'},
 '2025-04-29': {'1. open': '16857.00000',
  '2. high': '16857.00000',
  '3. low': '16753.00000',
  '4. close': '16754.00000'},
 '2025-04-28': {'1. open': '16824.00000',
  '2. high': '16863.00000',
  '3. low': '16819.00000',
  '4. close': '16849.00000'},
 '2025-04-25': {'1. open': '16799.00000',
  '2. high': '16879.00000',
  '3. low': '16799.00000',
  '4. close': '16824.00000'},
 '2025-04-24': {'1. open': '16875.00000',
  '2. high': '16884.00000',
  '3. low': '16849.00000',
  '4. close': '16864.00000'},
 '2025-04-23': {'1. open': '16857.00000',
  '2. high': '16884.00000',
  '3. low': '16844.00000',
  '4. close': '16859.00000'},
 '2025-04-22': {'1. open': '16803.00000',
  '2. high': '16864.00000',
  '3. low': '16799.00000',
  '4. close': 

In [3]:
# Konversi ke DataFrame
df = pd.DataFrame.from_dict(timeseries, orient='index')
df.columns = ['Open', 'High', 'Low', 'Close']
df.index = pd.to_datetime(df.index)
df = df.sort_index()
df = df.astype(float)
df

,Open,High,Low,Close
2014-11-07,12120.0,12170.0,12115.0,12135.0
2014-11-10,12170.0,12170.0,12135.0,12150.0
2014-11-11,12200.0,12200.0,12200.0,12200.0
2014-11-12,12200.0,12218.0,12142.0,12197.0
2014-11-13,12197.0,12210.0,12142.0,12200.0
...,...,...,...,...
2025-04-25,16799.0,16879.0,16799.0,16824.0
2025-04-28,16824.0,16863.0,16819.0,16849.0
2025-04-29,16857.0,16857.0,16753.0,16754.0
2025-04-30,16782.0,16782.0,16589.0,16594.0


In [4]:
# Buat salinan dan fitur tambahan
df_feat = df.copy()

# Fitur hari ini hanya Open
df_feat['Open_today'] = df_feat['Open']

# Fitur hari sebelumnya (lag 1)
df_feat['Close_t-1'] = df_feat['Close'].shift(1)
df_feat['Open_t-1'] = df_feat['Open'].shift(1)
df_feat['High_t-1'] = df_feat['High'].shift(1)
df_feat['Low_t-1'] = df_feat['Low'].shift(1)

# Hapus baris dengan missing values
df_feat.dropna(inplace=True)


In [5]:
# Pisahkan baris terakhir untuk prediksi real-time
latest_row = df_feat.iloc[[-1]]
df_feat = df_feat.iloc[:-1]

# Siapkan fitur dan target
X = df_feat[['Open_today', 'Close_t-1', 'Open_t-1', 'High_t-1', 'Low_t-1']]
y = df_feat['Close']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

In [6]:
# Lihat Train Data
X_train

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1
2014-11-10,12170.0,12135.0,12120.0,12170.0,12115.0
2014-11-11,12200.0,12150.0,12170.0,12170.0,12135.0
2014-11-12,12200.0,12200.0,12200.0,12200.0,12200.0
2014-11-13,12197.0,12197.0,12200.0,12218.0,12142.0
2014-11-14,12200.0,12200.0,12197.0,12210.0,12142.0
...,...,...,...,...,...
2023-03-20,15360.0,15360.0,15374.0,15415.0,15339.0
2023-03-21,15360.0,15354.0,15360.0,15375.0,15339.0
2023-03-22,15345.0,15339.0,15360.0,15360.0,15329.0
2023-03-23,15344.0,15339.0,15345.0,15345.0,15339.0


In [7]:
# Lihat Test Data
X_test

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1
2023-03-27,15185.0,15149.0,15345.0,15345.0,15129.0
2023-03-28,15170.0,15154.0,15185.0,15185.0,15149.0
2023-03-29,15085.0,15084.0,15170.0,15189.0,15059.0
2023-03-30,15059.0,15059.0,15085.0,15109.0,15054.0
2023-03-31,15060.0,15044.0,15059.0,15090.0,15039.0
...,...,...,...,...,...
2025-04-24,16875.0,16859.0,16857.0,16884.0,16844.0
2025-04-25,16799.0,16864.0,16875.0,16884.0,16849.0
2025-04-28,16824.0,16824.0,16799.0,16879.0,16799.0
2025-04-29,16857.0,16849.0,16824.0,16863.0,16819.0


In [8]:
# Buat model ElasticNet
model = ElasticNet(alpha=0.1, l1_ratio=0.5)
model.fit(X_train, y_train)

# Prediksi
y_pred = model.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.448e+06, tolerance: 1.197e+05
  model = cd_fast.enet_coordinate_descent(


In [9]:
# Evaluasi
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2 = r2_score(y_test, y_pred)

# Cetak metrik
print("Evaluation Metrics:")
print(f"MAE  : {mae:.4f}")
print(f"MSE  : {mse:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"MAPE : {mape:.2f}%")
print(f"R²   : {r2:.4f}")

Evaluation Metrics:
MAE  : 39.0235
MSE  : 2803.4897
RMSE : 52.9480
MAPE : 0.25%
R²   : 0.9898


In [10]:
# Visualisasi menggunakan Plotly
fig = go.Figure()

# Garis aktual
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_test.values,
    mode='lines',
    name='Actual',
    line=dict(width=2)
))

# Garis prediksi
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_pred,
    mode='lines',
    name='Predicted',
    line=dict(width=2),
))

fig.update_layout(
    title='ElasticNet Prediction of USD/IDR Closing Price',
    xaxis_title='Date',
    yaxis_title='Closing Price (IDR)',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white',
    width=1000,
    height=500
)

fig.show()

In [11]:
X_latest = latest_row[['Open_today', 'Close_t-1', 'Open_t-1', 'High_t-1', 'Low_t-1']]
X_latest

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1
2025-05-01,16674.0,16594.0,16782.0,16782.0,16589.0


In [13]:
# 🔮 Prediksi hari terakhir (yang tadi dipisahkan)

y_actual = latest_row['Close'].values[0]
y_forecast = model.predict(X_latest)[0]
absolute_error = abs(y_forecast - y_actual)

print("\nReal-time Forecast (Simulasi Hari Terakhir):")
print(f"Tanggal         : {latest_row.index[0].date()}")
print(f"Actual Close    : {y_actual:.2f}")
print(f"Predicted Close : {y_forecast:.2f}")
print(f"Absolute Error   : {absolute_error:.2f}")


Real-time Forecast (Simulasi Hari Terakhir):
Tanggal         : 2025-05-01
Actual Close    : 16597.00
Predicted Close : 16615.82
Absolute Error   : 18.82
